In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle


In [4]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [8]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag48h"] = df_bbidding1["market_price"].shift(96)
df_bbidding1.loc[:,"imbalance_price_lag48h"] = df_bbidding1["imbalance_price"].shift(96)
df_bbidding1.loc[:,"day_ahead_price_lag48h"] = df_bbidding1["day_ahead_price"].shift(96)
df_bbidding1.loc[:,"volume_lag48h"] = df_bbidding1["volume"].shift(96)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h
96,2021-02-22 23:00:00+00:00,39.86,40.74,850.50,47,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088636,...,1.146678e-09,2.732664e-20,40.952962,-0.658593,-4.207329,39.86000,26.46,32.75000,32.38,671.75
97,2021-02-22 23:30:00+00:00,25.88,40.74,755.65,48,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088686,...,1.146678e-09,2.732664e-20,53.022213,14.302050,2.706285,25.88000,29.81,62.50000,32.38,905.45
98,2021-02-23 00:00:00+00:00,31.95,35.25,821.25,1,1.000000,0.598181,-2.668594,1.327215e+00,-7.243929,...,1.336674e+00,-5.844249e+01,7.936466,44.084019,84.017522,31.95000,7.67,11.79015,17.34,781.05
99,2021-02-23 00:30:00+00:00,32.09,35.25,852.25,2,1.000000,0.598181,-83.386905,-4.611063e+01,-92.411183,...,1.028806e+00,-6.628035e+01,50.825184,23.217360,89.462875,32.18992,9.82,12.29000,17.34,694.45
100,2021-02-23 01:00:00+00:00,30.75,32.74,789.00,3,0.965926,0.598181,-92.621997,-3.236547e+01,-51.507430,...,5.531250e+01,4.985395e+01,94.284233,109.047937,123.758802,30.55000,10.40,16.88000,14.25,847.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,8.558464e+01,147.450968,...,2.575452e+02,3.120008e+02,389.209655,461.471346,565.748439,66.01000,0.00,70.00000,87.62,0.00
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,4.406274e+01,80.500728,...,1.529461e+02,2.133404e+02,285.856261,352.111989,494.824570,66.01000,0.00,72.00000,85.84,0.00
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-3.103038e+01,1.674412,...,8.055552e+01,1.386240e+02,177.132223,250.514362,333.223882,66.01000,0.00,71.00000,85.84,0.00
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-3.898848e+01,19.495393,...,7.219972e+01,1.286610e+02,157.102188,214.652927,345.416021,66.01000,83.73,98.00000,82.02,194.45


In [11]:
X = df_bbidding1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h","cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["imbalance_price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [ ]:
y_train = y_train.values
y_test = y_test.values

In [17]:
# Previous imports and data preprocessing remain the same
# Convert data to PyTorch tensors

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # Corrected this line
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create TensorDataset and DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

# Create TensorDataset and DataLoader for testing
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 128              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 10                  # Patience for early stopping
rel_improvement_threshold = 0.001  # Relative improvement threshold

In [18]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [19]:
import torch.nn.functional as F  # For MAE loss function

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Relative stopping
    rel_improvement = (previous_loss - epoch_loss) / previous_loss if previous_loss != 0 else 0
    if rel_improvement < rel_improvement_threshold:
        print(f"Relative improvement below threshold. Stopping training.")
        break
    previous_loss = epoch_loss

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
model.load_state_dict(best_model)

# Test the model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Loss: 154.6353
Epoch [2/500], Loss: 140.4911
Epoch [3/500], Loss: 131.4585
Epoch [4/500], Loss: 123.5438
Epoch [5/500], Loss: 116.6477
Epoch [6/500], Loss: 110.6413
Epoch [7/500], Loss: 105.4547
Epoch [8/500], Loss: 101.0868
Epoch [9/500], Loss: 97.5597


KeyboardInterrupt: 